## Importing Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import glob
import os

import librosa
import librosa.display


import torch
from torch import nn
from torchvision import models, transforms, datasets

from time import time
from tqdm import tqdm

## Defining Parameters

In [4]:
seed = 411
np.random.seed(seed)

path = "/Users/kemalcankucuk/Library/CloudStorage/OneDrive-KocUniversitesi/Okul/4. Sınıf/1. Dönem/COMP411/Project/comp411-project/gtzan_dataset"

path_audio_files = path + "/Users/kemalcankucuk/Library/CloudStorage/OneDrive-KocUniversitesi/Okul/4. Sınıf/1. Dönem/COMP411/Project/comp411-project/gtzan_dataset/genres_original"

path_imgs = "./mel_spectrogram_imgs/"

batch_size = 32

hop_length = 512

n_fft = 2048

device = 'cuda' if torch.cuda.is_available() else 'cpu'

genre_dict = {"blues":0,"classical":1,"country":2,"disco":3,"hiphop":4,"jazz":5,"metal":6,"pop":7,"reggae":8,"rock":9}

## Transforming Audio Files into Mel Spectograms and Saving the Images

In [13]:
print("Transforming the Audio Files into Mel Spectrograms:")

mel_spectogram_data = {}
for genre in genre_dict.keys():
    print("\t",genre)
    
    mel_spectogram_data[genre] = []

    for name in glob.glob(path_audio_files + genre + "/*"):
        
        if(name != "../input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00054.wav"):
        
            data,sampling_rate = librosa.load(name)

            mel_spec = librosa.feature.melspectrogram(y = data.ravel(), sr=sampling_rate,hop_length = hop_length)
            mel_spec_db = librosa.amplitude_to_db(mel_spec, ref=np.max)

            mel_spectogram_data[genre].append(mel_spec_db)
            

print("Saving the Mel Spectrogram Images:")
            
#os.mkdir(path_imgs)
path_imgs = "/Users/kemalcankucuk/Library/CloudStorage/OneDrive-KocUniversitesi/Okul/4. Sınıf/1. Dönem/COMP411/Project/comp411-project/gtzan_dataset/images_custom"
for genre in genre_dict.keys():
    print("\t",genre)
    try:
        os.mkdir(path_imgs + genre)
    except:
        pass
    
    for i in range(len(mel_spectogram_data[genre])):

        fig, ax = plt.subplots(1, figsize=(12,8))

        img = librosa.display.specshow(mel_spectogram_data[genre][i], sr = sampling_rate, hop_length = hop_length,cmap = 'cool',ax=ax)

        fig.savefig(path_imgs + genre + "/" + genre + "_" + str(i) + ".png")
        
        plt.close()

Transforming the Audio Files into Mel Spectrograms:
	 blues
	 classical
	 country
	 disco
	 hiphop
	 jazz
	 metal
	 pop
	 reggae
	 rock
Saving the Mel Spectrogram Images:
	 blues
	 classical
	 country
	 disco
	 hiphop
	 jazz
	 metal
	 pop
	 reggae
	 rock


In [14]:
import gc
gc.collect()

22

## Load and Transform the Data

In [15]:
%%time

# Define Tranforms
train_transforms = transforms.Compose([
    #transforms.Resize(224),
    transforms.ToTensor(),
    
    transforms.Normalize(mean=[0.4931, 0.9151, 0.9960], std=[0.4495, 0.1716, 0.0602])
])

test_transforms = transforms.Compose([
    #transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4931, 0.9151, 0.9960], std=[0.4495, 0.1716, 0.0602])
])

# Load the data
train_dataset = datasets.ImageFolder(path_imgs, transform = train_transforms)
val_dataset = datasets.ImageFolder(path_imgs, transform = test_transforms)
test_dataset = datasets.ImageFolder(path_imgs, transform = test_transforms)


torch.manual_seed(1)
num_train_samples = len(train_dataset)
#num_train_samples = 20000

# Permute the data
indices = torch.randperm(num_train_samples)

# Split the data into Train and Validation
train_testval_split = 0.2
train_split = int(num_train_samples * train_testval_split)
val_split = int(train_split * 0.5)

train_subset = torch.utils.data.Subset(train_dataset, indices[train_split:])
val_subset = torch.utils.data.Subset(val_dataset, indices[val_split:train_split])
test_subset = torch.utils.data.Subset(test_dataset, indices[:val_split])


print(f"Length of Train:{len(train_subset)}; Length of Val:{len(val_subset)}; Length of Test:{len(test_subset)}")



# Make DataLoaders 
train_dataloader = torch.utils.data.DataLoader(
    dataset=train_subset, 
    batch_size=batch_size,
    shuffle=True
)

val_dataloader = torch.utils.data.DataLoader(
    dataset=val_subset,
    batch_size=batch_size,
    shuffle=False
)

# Classes
classes = train_dataloader.dataset.dataset.classes

Length of Train:800; Length of Val:100; Length of Test:99
CPU times: user 26.7 ms, sys: 24.1 ms, total: 50.8 ms
Wall time: 92.3 ms


In [16]:
def mean_std(loader):
    images, lebels = next(iter(loader))
    # shape of images = [b,c,w,h]
    mean, std = images.mean([0,2,3]), images.std([0,2,3])
    return mean, std

mean, std = mean_std(train_dataloader)
print("mean and std: \n", mean, std)

mean and std: 
 tensor([ 0.6516, -2.1342, -5.4688]) tensor([0.6172, 2.3201, 4.8507])


## ResNet18 - Transfer Learning

In [17]:
# Load a Pretrained Model
resnet = models.resnet18(pretrained=True)

# Fix the trainable parameters
for parameter in resnet.parameters():
    parameter.requires_grad = False
    
    
# Number of Input Features in the Last Fully Connected Layer
in_features = resnet.fc.in_features

# Replacing the Last Fully Connected Layer
fc = nn.Linear(in_features=in_features, out_features=len(classes))
resnet.fc = fc


# Updating the Weights and Bias of the last layer
params_to_update = []
for name, param in resnet.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

# Define the Loss and Optimizer Functions
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params_to_update, lr=0.001)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/kemalcankucuk/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:08<00:00, 5.74MB/s]


## Training the Model

In [18]:
def train(model, criterion, optimizer, train_dataloader,test_dataloader,print_every,num_epoch):
    
    steps = 0
    train_losses, val_losses = [], []
    
    model.to(device)
    for epoch in tqdm(range(num_epoch)):
        running_loss = 0
        correct_train = 0
        total_train = 0
        start_time = time()
        iter_time = time()
        
        model.train()
        for i, (images, labels) in enumerate(train_dataloader):
            steps += 1
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            output = model(images)
            loss = criterion(output, labels)

            correct_train += (torch.max(output, dim=1)[1] == labels).sum()
            total_train += labels.size(0)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
            # Logging
            if steps % print_every == 0:
                print(f'Epoch [{epoch + 1}]/[{num_epoch}]. Batch [{i + 1}]/[{len(train_dataloader)}].', end=' ')
                print(f'Train loss {running_loss / steps:.3f}.', end=' ')
                print(f'Train acc {correct_train / total_train * 100:.3f}.', end=' ')
                with torch.no_grad():
                    model.eval()
                    correct_val, total_val = 0, 0
                    val_loss = 0
                    for images, labels in test_dataloader:
                        images = images.to(device)
                        labels = labels.to(device)
                        output = model(images)
                        loss = criterion(output, labels)
                        val_loss += loss.item()
                        
                        correct_val += (torch.max(output, dim=1)[1] == labels).sum()
                        total_val += labels.size(0)

                print(f'Val loss {val_loss / len(test_dataloader):.3f}. Val acc {correct_val / total_val * 100:.3f}.', end=' ')
                print(f'Took {time() - iter_time:.3f} seconds')
                iter_time = time()
                
                
                train_losses.append(running_loss / total_train)
                val_losses.append(val_loss / total_val)


        print(f'Epoch took {time() - start_time}') 
        torch.save(model, f'checkpoint_{correct_val / total_val * 100:.2f}')
        
    return model, train_losses, val_losses

In [19]:
print_every = 25
num_epoch = 100

resnet, train_losses, val_losses = train(
    model=resnet,
    criterion=criterion,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    test_dataloader=val_dataloader,
    print_every=print_every,
    num_epoch=num_epoch
)


plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1]/[100]. Batch [25]/[25]. Train loss 2.184. Train acc 22.750. Val loss 2.021. Val acc 36.000. Took 159.701 seconds
Epoch took 159.70740485191345


  1%|          | 1/100 [02:40<4:24:52, 160.53s/it]

Epoch [2]/[100]. Batch [25]/[25]. Train loss 0.935. Train acc 38.375. Val loss 1.747. Val acc 47.000. Took 152.739 seconds
Epoch took 152.7455699443817


  2%|▏         | 2/100 [05:55<4:50:25, 177.81s/it]


KeyboardInterrupt: 

## Testing

In [12]:
y_test = []
y_pred = []
for img, label in test_subset:
    img = torch.Tensor(img)
    img = img.to(device)
    resnet.eval()
    prediction = resnet(img[None])
    
    final_pred = classes[torch.max(prediction, dim=1)[1]]
    
    #print(label, genre_dict[final_pred])
    
    y_test.append(label)
    y_pred.append(genre_dict[final_pred])


In [16]:
print("Accuracy:",(100*(np.array(y_test) == np.array(y_pred)).sum()/len(y_test)))

Accuracy: 66.66666666666667
